# CreatorsHive Sコース昇格課題　ー機械学習編ー  
## 課題  
Titanicデータセットから機械学習を利用して，顧客が生存したかを推定しなさい．  
処理の流れは指定されたとおりに行いその実行コードと実行結果をJupyter-Notebookで保存して提出しなさい．  
## データセット内容  
今回利用するデータセットはTitanic号に載っていた顧客の情報とその顧客が事故後に生存したかが記載されているデータセットである．  
データのカラムは以下の通りである． 
* PassengerId：顧客ID  
* Survived：生存したかどうか
* Pclass:階級
* Name：氏名
* Sex：性別
* Age：年齢
* SibSp：一緒に乗船した兄弟・配偶者の人数
* Parch：一緒に乗船した親・子供の人数
* Ticket：チケット番号
* Fare：乗船料金
* Cabin：部屋番号
* Embarked：乗船した港（C = Cherbourg, Q = Queenstown, S = Southamton)
## 使用ライブラリ
* numpy
* pandas
* scikit-learn

## 1.1 データ読み込み  
pandas で train.csv を読み込み、読み込んだデータの先頭 5 行を表示しなさい．

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("train.csv")
print(df)
#type(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

## 1.2 前処理：不要なカラムの削除
読み込んだデータのカラムで前処理が難しいもの，もしくは生存者を推定する際に必要でないものを考え，削除しなさい．  
取り除いたデータの先頭5行を出力しなさい．  
例：Cabin（部屋番号）は特に規則性がなくまた、前処理が難しいため削除する．  

In [2]:
#前処理が難しいもの　Cabin
#不要なもの　PassengerId,Name,Fare,Embarked

In [3]:
del df['PassengerId']
del df['Name']
del df['Cabin']
del df['Fare']
del df['Embarked']

print(df[0:5])

   Survived  Pclass     Sex   Age  SibSp  Parch            Ticket
0         0       3    male  22.0      1      0         A/5 21171
1         1       1  female  38.0      1      0          PC 17599
2         1       3  female  26.0      0      0  STON/O2. 3101282
3         1       1  female  35.0      1      0            113803
4         0       3    male  35.0      0      0            373450


## 1.3 前処理:カテゴリデータの加工カラムの中には数値でないデータが存在する．  
そのデータを機械学習で利用できるように数値に変換をする．  

### 1.3.1 ラベルエンコーディング 
値が２種類のカラムを探しそのカラムの値を0,1に変換しなさい．  
処理後のデータの先頭5行を出力しなさい．  

In [4]:
df_treated = df.replace("male",0).replace("female",1)
print(df_treated[0:5])

   Survived  Pclass  Sex   Age  SibSp  Parch            Ticket
0         0       3    0  22.0      1      0         A/5 21171
1         1       1    1  38.0      1      0          PC 17599
2         1       3    1  26.0      0      0  STON/O2. 3101282
3         1       1    1  35.0      1      0            113803
4         0       3    0  35.0      0      0            373450


### 1.3.2 one-hotエンコーディング  
値が3種類以上ある場合、ラベルエンコーディングを適応させることはできない．  
one-hotエンコーディング，もしくはダミー変数化を利用して数値に変換しなさい．  
処理後のデータの先頭５行を出力しなさい．  

In [5]:
dummy_df = pd.get_dummies(df[['Ticket']])
df2 = pd.merge(df_treated, dummy_df, left_index=True, right_index=True)
print(df2[0:5])

   Survived  Pclass  Sex   Age  SibSp  Parch            Ticket  Ticket_110152  \
0         0       3    0  22.0      1      0         A/5 21171              0   
1         1       1    1  38.0      1      0          PC 17599              0   
2         1       3    1  26.0      0      0  STON/O2. 3101282              0   
3         1       1    1  35.0      1      0            113803              0   
4         0       3    0  35.0      0      0            373450              0   

   Ticket_110413  Ticket_110465        ...         Ticket_STON/O2. 3101290  \
0              0              0        ...                               0   
1              0              0        ...                               0   
2              0              0        ...                               0   
3              0              0        ...                               0   
4              0              0        ...                               0   

   Ticket_SW/PP 751  Ticket_W./C. 14258  Tic

## 1.4 前処理:欠損値の補完

データには欠損値（不明な値：NaN）が存在する．  
NaNが入っている顧客データを利用するには,NaNを数値に変換する必要がある．  
処理の手法としては  
 - その顧客データの削除  
 - 平均値を代入  
 - 他の変数から重回帰  

などが挙げられる．  
以上の手法の中で一つを利用して欠損値を補完しなさい．  

In [6]:
df3 = df2.fillna(df2['Age'].mean())
Age_rounded = round(df3['Age'])
del df3['Age']
df3['Age']=Age_rounded
#print(Age_rounded)

In [7]:
del df3['Ticket']
df3[0:5]

,Survived,Pclass,Sex,SibSp,Parch,Ticket_110152,Ticket_110413,Ticket_110465,Ticket_110564,Ticket_110813,...,Ticket_SW/PP 751,Ticket_W./C. 14258,Ticket_W./C. 14263,Ticket_W./C. 6607,Ticket_W./C. 6608,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735,Age
0,0,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22.0
1,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,38.0
2,1,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26.0
3,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35.0
4,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35.0


## 1.5 学習（決定木分析）
加工したデータを元にして学習を行い，生存者を推定する分類器を作成する．  

In [8]:
train_data = df3.values
xs = train_data[:, 1:] # Pclass以降の変数
y  = train_data[:, 0]  # 正解データ

print(xs[0:5])
print(y[0:5])

[[  3.   0.   1. ...,   0.   0.  22.]
 [  1.   1.   1. ...,   0.   0.  38.]
 [  3.   1.   0. ...,   0.   0.  26.]
 [  1.   1.   1. ...,   0.   0.  35.]
 [  3.   0.   0. ...,   0.   0.  35.]]
[ 0.  1.  1.  1.  0.]


データを学習用のデータを分類器が正しく推定しているかどうかを検証する検証用データの２つに分割しなさい．  
比率は  
学習用：検証用　= 80：20  
とする．  
その後学習用データを利用し学習しなさい．  

In [9]:
from sklearn.model_selection import train_test_split
xs_train,xs_test,y_train,y_test = train_test_split(xs,y,test_size=0.2,random_state=2)

forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(xs_train, y_train)

## 1.6 評価  
学習した分類器が正しく推定できているかどうかを確かめる．  
検証用データを利用して，精度（生存者推定の正答率）を出力しなさい．  

In [10]:
output = forest.predict(xs_test)
print(output)
print(y_test)

count = 0
length = len(y_test)
for i in range(length):
    if output[i] == y_test[i]:
        count += 1
rate = count / len(y_test) * 100

print("適合率は"+str(rate)+"%")


#print(len(test_data[:,0]), len(output))
#zip_data = zip(test_data[:,0].astype(int), output.astype(int))
#predict_data = list(zip_data)

[ 0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.  0.
  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  1.  1.  1.  1.  0.  0.  0.  1.  1.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  1.  0.  1.  1.  0.  1.  1.  1.
  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.
  0.  1.  0.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.
  0.  0.  1.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  1.  1.  1.  1.  1.  0.
  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.
  0.  1.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  

## 1.7 前処理の比較  
一通り行った前処理を改めて見直し，精度を上げるために改善できそうな点を考慮して，再度  
前処理，学習，評価を行いなさい．  
その際に工夫した点を記述しなさい．  

In [11]:
#乗船料金、乗船した港をデータに含んでいないが、関連性がありそうなので(ボートに近い部屋番号の人や、高級な部屋番号にいた人が生存確率が高い等）
#両方を含めて評価を行うことで精度が向上するのではないか。